In [1]:
import pandas as pd
from scipy.sparse import csr_matrix, hstack
import pickle

In [2]:
# Завантаження навчальної та тестової вибірок
train_d = pd.read_csv('data/train_sessions.csv', sep=',', index_col = 'session_id')
test_d = pd.read_csv('data/test_sessions.csv', sep=',', index_col = 'session_id')

In [3]:
# Приведення колонок 'time1' ,..., time10' до формату дати
times = [f'time{n}' for n in range(1,11)]
train_d[times] = train_d[times].apply(pd.to_datetime)
test_d[times] = test_d[times].apply(pd.to_datetime)

In [4]:
# Завантажуємо словник сайтів і переводим його у DataFrame
with open('data/site_dic.pkl', 'rb') as file:
    site_dict = pickle.load(file)
    
sities_dict = pd.DataFrame(list(site_dict.keys()), list(site_dict.values()), columns = ['site'])

In [5]:
print(f'Всього сайтів: {sities_dict.shape[0]} шт.')
sities_dict.head()

Всього сайтів: 48371 шт.


,site
25075,www.abmecatronique.com
13997,groups.live.com
42436,majeureliguefootball.wordpress.com
30911,cdt46.media.tourinsoft.eu
8104,www.hdwallpapers.eu


In [6]:
y_train = train_d['target'] # цільова змінна

In [7]:
# Повна таблиця, яка складається із навчальної і тестової ...
full_d = pd.concat([train_d.drop('target', axis = 1), test_d])

# ... розбивається через "idx_split"
idx_split = train_d.shape[0]

На початку час сесій не враховується. Натомість, береться до уваги частота сайтів, які полюбляє "Еліс".

In [8]:
sities = [f'site{n}' for n in range(1,11)]
full_sities = full_d[sities].fillna(0)
full_sities.head() # вибірка із індексів сайтів


,site1,site2,site3,site4,site5,site6,site7,site8,site9,site10
session_id,,,,,,,,,,
1,718,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,890,941.0,3847.0,941.0,942.0,3846.0,3847.0,3846.0,1516.0,1518.0
3,14769,39.0,14768.0,14769.0,37.0,39.0,14768.0,14768.0,14768.0,14768.0
4,782,782.0,782.0,782.0,782.0,782.0,782.0,782.0,782.0,782.0
5,22,177.0,175.0,178.0,177.0,178.0,175.0,177.0,177.0,178.0


Перевід таблиці id до розрідженого вигляду (оптимізованого під ML), де улюблені сайти "Еліс" замінені на "1", решта на "0"

In [9]:
# послідовність із індексами ("(flatten()" зжимає вектор до одновимірного)
sites_flatten = full_sities.values.flatten()

In [10]:
# шукана м-ця
full_sites_sparse = csr_matrix(([1] * sites_flatten.shape[0], sites_flatten, range(0, sites_flatten.shape[0] + 10, 10), ))[:,1:]

## 2. Побудова першої моделі (тільки за sities)

Використання логістичної регресії із пакету sklearn для роботи із 90% даних для навчання, і 10% - для валідації

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [12]:
# Розбиття вибірки на начальну і перевірочну
X_train_sparse = full_sites_sparse[: idx_split]
X_test_sparse = full_sites_sparse[idx_split :]

In [13]:
# Написання ф-ції, яка повертає якість моделі на відкладеній вибірці
def get_auc_lr_valid(X, y, C=1.0, ratio = .9, seed = 17 ):
    """
    X,y - вибірка,
    ratio - у якому відношенні поділяється вибірка,
    C, seed - кофіцієнт регуляризації, random state логістичної регресії
    """
    train_len = int(ratio * X.shape[0])
    X_train = X[: train_len, :]
    X_valid = X[train_len: , :]
    
    y_train = y[: train_len]
    y_valid = y[train_len :]
    
    logit = LogisticRegression(C = C, n_jobs = -1, random_state = seed)
    logit.fit(X_train, y_train)
    
    # ймовірність знаходження "Еліс" за прогнозом (valid_pred складається із двох ймовірностей - "0", "1"). Нас цікавить "1"
    valid_pred = logit.predict_proba(X_valid)[:, 1]
    
    # Вивід значення метрики roc_auc_score -- ймовірності за відкладеною вибіркою 
    return roc_auc_score(y_valid, valid_pred)
    

In [14]:
%%time
# Якість моделі на відкладеній вибірці
get_auc_lr_valid(X_train_sparse, y_train)

Wall time: 3.5 s


0.9642117625218563

Навчання моделі на тій же відкаленій вибірці (перед тим була задіяна лиш її частина).

In [15]:
# ф-ція дл запису кінцевого файлу на Kaggle
def write_to_subm(predicted_labels, out_file, target = 'target', index_label = 'session_id'):
    pred_df = pd.DataFrame(predicted_labels, index = np.arange(1, predicted_labels.shape[0] + 1), columns = [target])
    pred_df.to_csv(out_file, index_label = index_label)

In [16]:
%%time
# Тестовий запуск логістичної регресії та навчання вибірки
logit = LogisticRegression(n_jobs = -1, random_state = 17)
logit.fit(X_train_sparse, y_train)

# Тестовий прогноз за певними даними вибірки (перші 15 рядків, усі колонки). На виході - ймоівності "1", тобто виявлення "Еліс"
test_pred = logit.predict_proba(X_test_sparse)[:, 1]
test_pred

Wall time: 3.42 s


array([2.21646366e-03, 2.51945472e-09, 6.16240154e-09, ...,
       8.40844944e-03, 3.86231242e-04, 1.29356810e-05])

In [17]:
pd.Series(test_pred, index = range(1, test_pred.shape[0] + 1), 
          name = 'target').to_csv('data/banchmark_1.csv', header = True, index_label = 'session_id')

In [18]:
# Якість моделі на повній вибірці
# 0.90736

## Покращення моделі побудовою нових ознак

Використання моделі часу РРРРММ для щосмісячного лінійного тренду всього періоду

In [19]:
from sklearn.preprocessing import StandardScaler
from scipy.sparse import csr_matrix, hstack

In [20]:
# Створюютя нові таблиці ознак
new_feat_train = pd.DataFrame(index = train_d.index)
new_feat_test = pd.DataFrame(index = test_d.index)

# Переведення часу у зручний формат
new_feat_train['year_month'] = train_d['time1'].apply(lambda ts: 100 * ts.year + ts.month)
new_feat_test['year_month'] = test_d['time1'].apply(lambda ts: 100 * ts.year + ts.month)

new_feat_train.shape, new_feat_test.shape

((253561, 1), (82797, 1))

In [21]:
# маштабування ознак
scaler = StandardScaler()
scaler.fit(new_feat_train['year_month'].values.reshape(-1,1))
scaler.fit(new_feat_test['year_month'].values.reshape(-1,1))

new_feat_train['year_month_scaled'] = scaler.transform(new_feat_train['year_month'].values.reshape(-1,1))
new_feat_test['year_month_scaled'] = scaler.transform(new_feat_test['year_month'].values.reshape(-1,1))

Після маштабування знову використовуємо ROC AUC:

In [22]:
X_train_sparse_new = csr_matrix(hstack([X_train_sparse, new_feat_train['year_month_scaled'].values.reshape(-1,1)]))
X_test_sparse_new = csr_matrix(hstack([X_test_sparse, new_feat_test['year_month_scaled'].values.reshape(-1,1)]))

In [23]:
%%time
# Перевірка якості нової моделі
get_auc_lr_valid(X_train_sparse_new, y_train)

Wall time: 3.38 s


0.9465132827324478

In [24]:
%%time
# Тестовий запуск логістичної регресії та навчання вибірки
logit = LogisticRegression(n_jobs = -1, random_state = 17)
logit.fit(X_train_sparse_new, y_train)

# Тестовий прогноз за певними даними вибірки (перші 15 рядків, усі колонки). На виході - ймоівності "1", тобто виявлення "Еліс"
test_pred = logit.predict_proba(X_test_sparse_new)[:, 1]
test_pred

Wall time: 3.99 s


array([2.86846751e-03, 3.47183323e-10, 7.69118419e-10, ...,
       6.15475057e-03, 7.43739408e-04, 6.29254653e-06])

In [25]:
pd.Series(test_pred, index = range(1, test_pred.shape[0] + 1), 
          name = 'target').to_csv('data/banchmark_2.csv', header = True, index_label = 'session_id')

In [26]:
# Якість моделі на повній вибірці
# 0.90882